# The University of Hong Kong
## DASC7600 Data Science Project 2024
## Discrete Compartmental Model

# Import Modules and Settings

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from scipy.signal import find_peaks, peak_widths

import covid_module
from discrete_compartmental_model import *

# Settings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
# Read csv file
covid_hk_case_std = pd.read_csv('./data/std_data/hk/covid_hk_case_std.csv')

# New Case Counts

In [3]:
covid_hk_new_case_cnt_df = covid_module.get_date_count(covid_hk_case_std, 'report_date', '%Y%m%d')

# Fitting Model - First Wave

In [4]:
# New case counts for 1st wave
first_wave_new_case_cnt = covid_hk_new_case_cnt_df["count"].values[43:89].tolist()

In [5]:
# Discrete_SIR_model_first_wave = Discrete_SIR_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt[1:]), first_wave_new_case_cnt[0], 0])
# plot_compartmental_model_result(Discrete_SIR_model_first_wave.fit(), first_wave_new_case_cnt, Discrete_SIR_model_first_wave.classes)

In [6]:
# Discrete_SIS_model_first_wave = Discrete_SIS_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt[1:]), first_wave_new_case_cnt[0]])
# plot_compartmental_model_result(Discrete_SIS_model_first_wave.fit(), first_wave_new_case_cnt, Discrete_SIS_model_first_wave.classes)

In [7]:
# Discrete_SIRS_model_first_wave = Discrete_SIRS_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt[1:]), first_wave_new_case_cnt[0], 0])
# plot_compartmental_model_result(Discrete_SIRS_model_first_wave.fit(), first_wave_new_case_cnt, Discrete_SIRS_model_first_wave.classes)

In [8]:
# Discrete_SEIR_model_first_wave = Discrete_SEIR_model(first_wave_new_case_cnt,
#                                                      [sum(first_wave_new_case_cnt[1:]), first_wave_new_case_cnt[0]/2, first_wave_new_case_cnt[0]/2, 0])
# plot_compartmental_model_result(Discrete_SEIR_model_first_wave.fit(), first_wave_new_case_cnt, Discrete_SEIR_model_first_wave.classes)

# Wave Detection

In [9]:
new_case_cnt = covid_hk_new_case_cnt_df['count']
wave_peaks, _ = find_peaks(new_case_cnt, height=0, width=5) # Return: peaks (ndarray), properties (dict)
wave_widths = peak_widths(new_case_cnt, wave_peaks, rel_height=1) # Return: widths (ndarray), width_heights (ndarray), left_ips (ndarray), right_ips (ndarray)

In [10]:
# # Plot the new case counts with different waves identified
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(new_case_cnt)
# ## x-axis
# plt.plot(np.zeros_like(new_case_cnt), '--', color='gray')
# ## Peak of each wave
# plt.plot(wave_peaks, new_case_cnt[wave_peaks], 'x', color='green')
# ## Period of each wave
# plt.plot(wave_widths[2], wave_widths[1], '|', color='g')
# plt.plot(wave_widths[3], wave_widths[1], '|', color='g')
# plt.hlines(*wave_widths[1:], color='g')
# ## Title, x-axis label, y-axis label
# plt.title('Number of Covid-19 cases and waves identified (Hong Kong)')
# plt.xlabel('Days')
# plt.ylabel('Count')
# plt.show()

# Fitting SIR Model on Detected Waves

In [11]:
nbr_wave = len(wave_widths[0])

wave_index_list = []
wave_predicted_cnt_list = []
for i in range(nbr_wave):
    wave_start_index = int(wave_widths[2][i])+1
    wave_end_index = int(wave_widths[3][i])
    wave_new_case_cnt = covid_hk_new_case_cnt_df["count"].values[wave_start_index:wave_end_index].tolist()
    Discrete_SIR_model_hk = Discrete_SEIR_model(wave_new_case_cnt,
                                                [sum(wave_new_case_cnt[1:]), wave_new_case_cnt[0]/2, wave_new_case_cnt[0]/2, 0])
    wave_predicted_cnt = Discrete_SIR_model_hk.fit()[1]
    wave_index_list.append(range(wave_start_index, wave_end_index))
    wave_predicted_cnt_list.append(wave_predicted_cnt)

The optimal parameters are: [5.40538030e-04 8.60487927e-01 1.02100332e-01]
The optimal parameters are: [8.42221671e-05 7.20361529e-01 1.41394342e-01]
The optimal parameters are: [0.00061599 0.01281842 0.15153177]
The optimal parameters are: [0.00116735 0.49535694 0.01046545]


In [12]:
nbr_wave = len(wave_widths[0])

wave_index_list = []
wave_predicted_cnt_list = []
for i in range(nbr_wave):
    wave_start_index = int(wave_widths[2][i])+1
    wave_end_index = int(wave_widths[3][i])
    wave_new_case_cnt = covid_hk_new_case_cnt_df["count"].values[wave_start_index:wave_end_index].tolist()
    Discrete_SIR_model_hk = Discrete_SEIR_model(wave_new_case_cnt,
                                                [sum(wave_new_case_cnt[1:]), wave_new_case_cnt[0], wave_new_case_cnt[0], 0])
    wave_predicted_cnt = Discrete_SIR_model_hk.fit()[1]
    wave_index_list.append(range(wave_start_index, wave_end_index))
    wave_predicted_cnt_list.append(wave_predicted_cnt)

The optimal parameters are: [4.86479231e-04 8.41590364e-01 1.05363404e-01]
The optimal parameters are: [7.90838080e-05 7.30233728e-01 1.43452088e-01]
The optimal parameters are: [0.00075779 0.01113818 0.12320952]
The optimal parameters are: [0.000778   0.59396067 0.04215844]


In [13]:
# # Plot the new case counts with different waves identified and their predicted counts
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(new_case_cnt)
# ## x-axis
# plt.plot(np.zeros_like(new_case_cnt), '--', color='gray')
# ## Peak of each wave
# plt.plot(wave_peaks, new_case_cnt[wave_peaks], 'x', color='green')
# ## Period of each wave
# plt.plot(wave_widths[2], wave_widths[1], '|', color='green')
# plt.plot(wave_widths[3], wave_widths[1], '|', color='green')
# plt.hlines(*wave_widths[1:], color='green')
# ## Title
# plt.title('Number of Covid-19 cases and waves identified (Hong Kong)')
# plt.xlabel('Days')
# plt.ylabel('Count')
# # Predicted new case counts counts
# for i in range(nbr_wave):
#     plt.plot(wave_index_list[i], wave_predicted_cnt_list[i], color='red')
# plt.show()